## Value Iteration

Value iteration is similar to policy iteration in that it is applied in a Markov Decision Process setting and follows the dynamic programming paradigm, although this time over a value space by applying a contraction operator known as the Bellman backup operator in every iterative step of the algorithm.
It builds up on the theorem that the Bellman backup operator is a contraction operator, so applying it iteratively guarantees the convergence of the value functions to a global optimum.
Value iteration is also rarely used in practice, since it requires full knowledge of all states and transition dynamics, which are usually not given to us.
Nevertheless, like policy iteration this algorithm is a classic, so it is worth studying.

### Characteristics of Value Iteration:

##### Model based
Value iteration is model based, i.e. it requires full knowledge of all states, as well as transition dynamics.
Notice that in our case, when trying to solve the simplified Grid World, the transition dynamics are deterministic and we do not need to keep track of transition probabilities.

##### Finite and discrete state and action spaces
In order for value iteration to work, the environment has to have a finite state and action space, because it saves a model of the environment internally, i.e. it saves state values in a table internally.
The aforementioned is only possible if the state and action spaces are finite and discrete.

##### Dynamic programming
Value iteration can be categorized as a case of dynamic programming optimization method.
Dynamic programming leverages *bootstrapping* to help us get value estimates with only one backup.
The reason we are able to backup over just one transition in dynamic programming is because we leverage the *Markovian assumption* of the domain.

##### Convergence Theorem
Since Bellman optimality operator is a contraction operator, value function iteration converges to a *global optimum* of the mean-squared error (MSE) for both finite and infinite horizon settings.

##### Bellman optimality backup operator
The Bellman optimality backup operator is used during value iteration to calculate the expected future sum of rewards for a given state.
The Bellman optimality operator updates the state value with the maximal state-action value.
This is different from the Bellman expectation operator used in policy iteration, where the state values were weighted among the best actions of the policy.
Nevertheless, in most cases they **deliver the same results**.

##### Discount factor
The discount factor must take a value in the range $[0...1]$.
 - setting it to $1$ means that we put as much value to future states as the current state.
 - setting it to $0$ means that we do not value future states at all, only the current state.

##### Initialization
For value iteration we keep track only of state value functions in a table. State values are initialized to 0 for all states.


In [1]:
from environment import GraphicDisplay, Env

class ValueIteration:
    def __init__(self, env):
        self.env = env
        # 2-d list for the value function
        self.value_table = [[0.0] * env.width for _ in range(env.height)]
        self.discount_factor = 0.9

### Value Iteration

Value Iteration iteratively updates and improves state values, until the previous table of state values and the new table of state values are the same.
Only after the state values converge can we extract the policy.

Given a state $s$, it calculates the expected future sum of rewards for that state via the following formula:

$V'(s) \gets \max_{k=1}^n \{r(s, a_k) + \gamma \sum_{q=1}^m P(s_q | s, a_k) V(s_{q})\}$

where:
 - $s$ - current state
 - $s_{q}$ - possible next state when action $a$ is taken in state $s$
 - $π(s, a_k)$ - probability of taking action $a_k$ in state $s$ according to policy $π$
 - $r(s, a_k)$ - reward from the environment after taking action $a_k$ in state $s$
 - $\gamma$ - discount factor
 - $P(s_q | s, a_k)$ - transition probability to state $s_q$ when action $a_k$ is taken in state $s$
 - $V'(s)$ - updated value function given state $s$
 - $V(s_{q})$ - current value function given next state

In our example though, the environment is deterministic, not stochastic so we abstain from the usage of transition probabilities $P(s_q | s, a_k)$ in our implementation.

In [2]:
class ValueIteration(ValueIteration):

    # get next value function table from the current value function table
    def value_iteration(self):
        next_value_table = [[0.0] * self.env.width
                                    for _ in range(self.env.height)]
        for state in self.env.get_all_states():
            if state == [2, 2]:
                next_value_table[state[0]][state[1]] = 0.0
                continue
            value_list = []

            for action in self.env.possible_actions:
                next_state = self.env.state_after_action(state, action)
                reward = self.env.get_reward(state, action)
                next_value = self.get_value(next_state)
                value_list.append((reward + self.discount_factor * next_value))
            # return the maximum value(it is the optimality equation!!)
            next_value_table[state[0]][state[1]] = round(max(value_list), 2)
        self.value_table = next_value_table

##### Multiple global optima

Notice that in the case of policy evaluation, the state value $V(s)$ is calculated over all possible actions of that state, weighted by the probability of taking that action from the policy.
As we are going to see below, the policy is a uniform distribution over actions that reveal the maximal expected future sums of rewards, i.e. value states of next states.
This means that the policy evaluation step ensures the bootstrapping of multiple optimal solutions if there exist many of them, i.e. alternative solutions that are also global optima do not get lost.

### Other methods

##### Helper methods

In [3]:
class ValueIteration(ValueIteration):
    # get action according to the current value function table
    def get_action(self, state):
        action_list = []
        max_value = -99999

        if state == [2, 2]:
            return []

        # calculating q values for the all actions and
        # append the action to action list which has maximum q value
        for action in self.env.possible_actions:

            next_state = self.env.state_after_action(state, action)
            reward = self.env.get_reward(state, action)
            next_value = self.get_value(next_state)
            value = (reward + self.discount_factor * next_value)

            if value > max_value:
                action_list.clear()
                action_list.append(action)
                max_value = value
            elif value == max_value:
                action_list.append(action)

        return action_list

    def get_value(self, state):
        return round(self.value_table[state[0]][state[1]], 2)

##### Inferring the policy

Inferring the policy is the last step of the value iteration algorithm.
After executing the value iteration multiple times, the policy can be inferred from the state values in the `get_action()` method.
In each step we select the action leading to the maximal state value.

##### Main function

In [4]:
if __name__ == "__main__":
    env = Env()
    value_iteration = ValueIteration(env)
    grid_world = GraphicDisplay(value_iteration)
    grid_world.mainloop()

## Results

<h3 style="text-align:center">Initially</h3>
<img src="ipynb_results/initial.png" alt="initial.png" width="50%" />

<h3 style="text-align:center">Midway</h3>
<img src="ipynb_results/midway.png" alt="midway.png" width="50%" />

<h3 style="text-align:center">Converged</h3>
<img src="ipynb_results/converged.png" alt="converged.png" width="50%" />

<h3 style="text-align:center">Inferring policy</h3>
<img src="ipynb_results/converged_policy_wa.png" alt="converged_policy_wa.png" width="50%" />

<h3 style="text-align:center">Final</h3>
<img src="ipynb_results/final.png" alt="final.png" width="50%" />